We are now going to learn about mixture models; models that combine pieces of several different distributions in intereting ways. These types of models can help with over-dispersion, zero-inflation, and ordered categories.

In [1]:
import sys, re, warnings

import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.special import factorial
import matplotlib.pyplot as plt
import torch
tt =  torch.tensor
from torch.distributions import TransformedDistribution
import pyro
from pyro.distributions import (
    BetaBinomial,
    Binomial,
    Categorical,
    Delta,
    Dirichlet,
    Exponential,
    Gamma,
    GammaPoisson,
    Normal,
    Poisson,
    TorchDistribution,
    TransformedDistribution,
    ZeroInflatedPoisson,
    OrderedLogistic
)
from torch.distributions import Distribution
Distribution.set_default_validate_args(False)

from pyro.distributions.transforms import Transform
from pyro.ops.stats import hpdi, waic, resample
from pyro.infer import Predictive
from pyro.distributions import constraints
import arviz as az 
from utils import train_nuts, unnest_samples, traceplot, precis, plot_intervals

torch.multiprocessing.set_sharing_strategy("file_system")

In [2]:
%load_ext watermark
%watermark -n -u -v -iv -w

Last updated: Sun Sep 15 2024

Python implementation: CPython
Python version       : 3.9.19
IPython version      : 8.18.1

torch     : 2.3.0
arviz     : 0.17.1
sys       : 3.9.19 (main, May  6 2024, 19:43:03) 
[GCC 11.2.0]
scipy     : 1.12.0
pyro      : 1.9.1
re        : 2.2.1
matplotlib: 3.9.0
numpy     : 1.26.4
pandas    : 2.2.2

Watermark: 2.4.3



### Code 12.12 - 12.29
Now we will discuss ordered categorical models. This type of model arises whenever we have a categorical variable that doesn't have a direct numerical interpretation, but nevertheless does have some sort of order (e.g. "worst" < "bad" < "good" < "better" < "best"). We will analyze this in the context of "trolley problems", where someone is placed in a hypothetical situation involving a runaway trolley which will end up killing various people depending on the choices made.

In [3]:
trolley_df = pd.read_csv("data/Trolley.csv", sep=";")
print(len(trolley_df), "rows")
trolley_df[["story", "age", "male", "edu", "action", "intention", "contact", "response"]].sample(6)

9930 rows


,story,age,male,edu,action,intention,contact,response
4146,aqu,43,0,Graduate Degree,1,0,0,4
5454,che,38,1,Some College,0,1,0,7
3892,spe,37,0,Some College,1,1,0,1
7993,boa,57,1,Graduate Degree,0,0,0,3
8828,box,21,1,Some College,1,0,0,3
6108,box,28,0,Some College,1,1,0,4


In order to enforce the ordering of the response variables (so that increasing one of the associated predictor variables increases the response), we will use a cumulative link function. Basically, each outcome level gets its own "intercept" $\alpha_k$ that is related to the cumulative probability via
$$
\log\left(\frac{Pr(y_i \le k)}{1 - Pr(y_i \le k)}\right) = \alpha_k
$$
You will notice that this is the logit function, so its inverse is the sigmoid $Pr(y_i \le k) = \sigma(\alpha_k) = e^{\alpha_k}/(1+e^{\alpha_k})$. The idea is that as we introduce predictor variables, we add them onto $\alpha_k$ as a linear model, then do the sigmoid thing to convert them into cumulative

In [4]:

from ipywidgets import *
from IPython.display import display, clear_output

x, y = np.unique(trolley_df["response"], return_counts=True)
y = np.cumsum(y)/y.sum()
x = np.log(y/(1-y))
title = r"$ \log \frac{P(y_i \leq k)}{1 - P(y_i \leq k)} = \alpha_k + \phi_i$"
font = {'family': 'serif',
        'color':  'dodgerblue',
        'weight': 'normal',
        'size': 12,
        }
plt.ion()

def plot_cumulative_logit(ϕ):
    global x
    clear_output(wait=True)
    fig, axes = plt.subplots(ncols=2, figsize=(10, 5))
    x_minus_phi = x + ϕ
    y = torch.sigmoid(tt(x_minus_phi))
    # [0.0514, 0.0939, 0.1520, 0.3203, 0.4725, 0.6834, 1.0000]
    plt.sca(axes[0])
    plt.stem(x_minus_phi, y, linefmt='--')
    plt.ylabel("cumulative proportion")
    plt.xlabel("log-cumulative odds")
    plt.xlim(-6, 6)
    plt.vlines(ϕ, 0, 1.1, ls="-", color="k")
    for idx in range(len(x_minus_phi)-1):
        plt.hlines(y[idx], -6, x_minus_phi[idx],  ls="--")
        plt.title(title)
    for idx in range(len(x_minus_phi[:-1])):
        plt.text(x_minus_phi[idx], -0.05, rf"$\alpha_{idx+1}$", fontdict=font)
    plt.sca(axes[1])
    pk = []
    for idx in range(len(x_minus_phi)-1):
        idxx = len(x_minus_phi) - 1 - idx
        val = y[idxx].item() - y[idxx-1].item()
        pk.append(val)
    pk.append(y[0].item())
    pk = pk[::-1]
    pk = np.array(pk)
    xx = [x+1 for x in range(len(pk))]
    plt.bar(xx, pk)
    plt.ylabel("probability")
    plt.xlabel("Observed Values")
    plt.ylim(0, 0.7)
    #plt.show()
    plt.draw()
    plt.pause(0.0001)
    plt.clf()
    #fig.canvas.draw()
    #fig.canvas.flush_events()
    display(fig)
    plt.close()
    

interact(plot_cumulative_logit, ϕ=widgets.FloatSlider(value=-1, min=-2, max=2, step=0.1))


/tmp/ipykernel_14226/752784804.py:6: RuntimeWarning: divide by zero encountered in divide
  x = np.log(y/(1-y))


interactive(children=(FloatSlider(value=-1.0, description='φ', max=2.0, min=-2.0), Output()), _dom_classes=('w…

<function __main__.plot_cumulative_logit(φ)>

In [5]:
'''
from ipywidgets import *

x, y = np.unique(trolley_df["response"], return_counts=True)
y = np.cumsum(y)/y.sum()
x = np.log(y/(1-y))

def plot_cumulative_logit(i):
    global x
    x_minus_phi = x - i
    y = torch.sigmoid(tt(x_minus_phi))
    plt.stem(x_minus_phi + i, y, linefmt='--')
    plt.ylabel("cumulative proportion")
    plt.xlabel("log-cumulative odds")
    plt.xlim(-6, 6)
    plt.vlines(i, 0, 1.1, ls="-", color="k")
    for idx in range(len(x_minus_phi)-1):
        plt.hlines(y[idx], -6, x_minus_phi[idx] + i,  ls="--")
    plt.show()

interact(plot_cumulative_logit, i= widgets.FloatSlider(value=0, min=-2, max=2, step=0.2))
'''

'\nfrom ipywidgets import *\n\nx, y = np.unique(trolley_df["response"], return_counts=True)\ny = np.cumsum(y)/y.sum()\nx = np.log(y/(1-y))\n\ndef plot_cumulative_logit(i):\n    global x\n    x_minus_phi = x - i\n    y = torch.sigmoid(tt(x_minus_phi))\n    plt.stem(x_minus_phi + i, y, linefmt=\'--\')\n    plt.ylabel("cumulative proportion")\n    plt.xlabel("log-cumulative odds")\n    plt.xlim(-6, 6)\n    plt.vlines(i, 0, 1.1, ls="-", color="k")\n    for idx in range(len(x_minus_phi)-1):\n        plt.hlines(y[idx], -6, x_minus_phi[idx] + i,  ls="--")\n    plt.show()\n\ninteract(plot_cumulative_logit, i= widgets.FloatSlider(value=0, min=-2, max=2, step=0.2))\n'